# Taproot Analytics Project

## Step 1
First we need to import the libraries we're working with, set up the directory, and read in the data.
Not all of the data is useful to us for the purposes of answering business questions, so we're only focusing on a few datasets.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sklearn
path = 'C:\\Users\\lucky\\Teradata Files'

In [2]:
Prjt_Cat = os.path.join(path, 'project_categories.csv')
Prjt_Cat = pd.read_csv(Prjt_Cat)
Prjts = os.path.join(path, 'project_export.csv')
Prjts = pd.read_csv(Prjts)
User_Inqry = os.path.join(path, 'project_inquiries.csv')
User_Inqry = pd.read_csv(User_Inqry)
Session = os.path.join(path, 'session_export.csv')
Session = pd.read_csv(Session)

### Questions to answer are:
    Which project types produce the best outcomes in survey data?
    Which project types perform better on the platform? Match rate, days to match, # of inquiries received.
    Which project types take the longest to complete? Shortest?

# Step 2
Next let's take a quick glance at the data and see what we have to work with, what data is important for us, and what needs cleaning at first glance to do some exploratory data analysis.

# Prjt_Cat

In [3]:
#exploring Prjt_Cat dataset
Prjt_Cat.head(10)

,id,created_at,updated_at,group_slug,enabled,international,name,slug
0,1,2016-04-04 21:22:07.650464,2016-04-04 21:22:07.650464,business,t,f,Accounting & Finance,accounting
1,3,2016-04-04 21:22:08.185673,2016-04-04 21:22:08.185673,business,t,t,Evaluation,evaluation
2,5,2016-04-04 21:22:08.297166,2016-04-04 21:22:08.297166,business,t,t,Program Design,program
3,6,2016-04-04 21:22:08.365836,2016-04-04 21:22:08.365836,business,t,t,Research,research
4,7,2016-04-04 21:22:08.425545,2016-04-04 21:22:08.425545,hr,t,f,HR Management,management
5,8,2016-04-04 21:22:08.509798,2016-04-04 21:22:08.509798,hr,t,f,Board Development,board
6,9,2016-04-04 21:22:08.564899,2016-04-04 21:22:08.564899,hr,t,t,Staff Development,staff
7,11,2016-04-04 21:22:08.716657,2016-04-04 21:22:08.716657,marketing,t,f,Brand Development,brand
8,12,2016-04-04 21:22:08.766127,2016-04-04 21:22:08.766127,marketing,t,f,Messaging,messaging
9,13,2016-04-04 21:22:08.829061,2016-04-04 21:22:08.829061,marketing,t,t,Multimedia,multimedia


### Thoughts on this dataset
This seems pretty clean, but we don't need all of these columns. Let's drop the created, updated, and slug columns to reduce the size of our table. 
We'll mostly be using this dataset to provide context to the projects we'll be analyzing.

In [4]:
Prjt_Cat = Prjt_Cat.drop(labels=['created_at','updated_at','slug', 'enabled', 'international'],axis=1)
print(Prjt_Cat.sort_values(by='id',axis=0))

    id group_slug                       name
0    1   business       Accounting & Finance
21   2   business          Business Planning
1    3   business                 Evaluation
18   4   business         Project Management
2    5   business             Program Design
3    6   business                   Research
4    7         hr              HR Management
5    8         hr          Board Development
6    9         hr          Staff Development
17  10  marketing         Marketing Strategy
7   11  marketing          Brand Development
8   12  marketing                  Messaging
9   13  marketing                 Multimedia
10  14  marketing                     Design
11  15  marketing       Copy writing/editing
12  16  marketing           Public Relations
20  17         it          IT Infrastructure
13  18         it                        CRM
14  19         it             Website design
19  20         it        Website development
15  21         it         Mobile Development
16  22   b

In [5]:
Prjt_Cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          22 non-null     int64 
 1   group_slug  22 non-null     object
 2   name        22 non-null     object
dtypes: int64(1), object(2)
memory usage: 656.0+ bytes


# Prjts

In [6]:
# Exploring Prjts dataset
Prjts.head()

,id,organization_id,description,created_at,updated_at,state,user_id,needs_accomplish,needs_support,needs_value,...,success_story,partner_organization_id,match_job_id,satisfaction_rating,agreed_at_community,timeline,publish_externally,enable_success_story,is_archived,share_metadata
0,5948,4809,Website design optimization for new brand,2017-07-06 17:27:44.905846,2017-08-30 01:39:47.519662,closed,148960,We are looking for a web designer who can assi...,Shaw is the most rapidly gentrifying zip code ...,We have a marketing & branding expert who has ...,...,NaN,NaN,NaN,NaN,2017-07-06 17:54:28.281227,2,t,f,f,"{""project_id"": null}"
1,5908,2051,New logo design,2017-06-28 16:55:47.85256,2017-11-11 18:17:38.379281,completed,122660,We are looking for a new Visual identity (logo...,We are a 25+ year organization that could use ...,The time is NOW since we have concluded a bran...,...,NaN,NaN,NaN,NaN,2016-04-14 21:25:00.799927,2,t,f,f,"{""project_id"": null}"
2,9339,1401,NaN,2019-01-15 00:00:34.636008,2019-01-15 00:00:34.636008,draft,169620,Our brochure needs revision and slight redesig...,We are opening two new clinics this year and o...,Our existing brochure is already in English an...,...,NaN,NaN,NaN,NaN,NaN,2,t,f,f,"{""project_id"": null}"
3,5975,4831,LOI and grant application,2017-07-10 15:36:19.184276,2017-09-27 13:39:17.766539,closed,149358,We are looking for a grant writer to assist us...,NARP has been around since the late 1960s and ...,We have identified our target grantor and have...,...,NaN,NaN,NaN,NaN,2017-07-26 16:09:05.920907,2,t,f,f,"{""project_id"": null}"
4,9981,7860,NaN,2019-04-03 03:19:57.84851,2019-04-03 03:19:57.84851,draft,172963,Organizing books and prepare tax documents. We...,Optimistic Movement was founded in June 2017. ...,We have purchased Quickbooks. Most of expenses...,...,NaN,NaN,NaN,NaN,NaN,0,t,f,f,"{""project_id"": null}"


In [7]:
Prjts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11549 entries, 0 to 11548
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       11549 non-null  int64  
 1   organization_id          11549 non-null  int64  
 2   description              9874 non-null   object 
 3   created_at               11549 non-null  object 
 4   updated_at               11549 non-null  object 
 5   state                    11549 non-null  object 
 6   user_id                  11549 non-null  int64  
 7   needs_accomplish         11547 non-null  object 
 8   needs_support            10830 non-null  object 
 9   needs_value              10817 non-null  object 
 10  campaign_id              455 non-null    float64
 11  image_id                 11507 non-null  float64
 12  project_inquiries_count  11549 non-null  int64  
 13  admin_id                 9313 non-null   float64
 14  project_group         

### Thoughts on this dataset:
1) First thing I noticed is we have a lot of null values we'll have to grapple with. We can either do imputation or remove observations with nulls. 
2) We also need to create a new column 'days_to_update' from the 'updated_at' column less the 'created_at' column. 
3) I think there are a few columns in this dataset we can drop as well

In [8]:
Prjts['created_at'] = pd.to_datetime(Prjts['created_at'], format="%Y-%m-%d %H:%M:%S.%f")
Prjts['updated_at'] = pd.to_datetime(Prjts['updated_at'], format="%Y-%m-%d %H:%M:%S.%f")
Prjts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11549 entries, 0 to 11548
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       11549 non-null  int64         
 1   organization_id          11549 non-null  int64         
 2   description              9874 non-null   object        
 3   created_at               11549 non-null  datetime64[ns]
 4   updated_at               11549 non-null  datetime64[ns]
 5   state                    11549 non-null  object        
 6   user_id                  11549 non-null  int64         
 7   needs_accomplish         11547 non-null  object        
 8   needs_support            10830 non-null  object        
 9   needs_value              10817 non-null  object        
 10  campaign_id              455 non-null    float64       
 11  image_id                 11507 non-null  float64       
 12  project_inquiries_count  11549 n

In [9]:
Prjts['days_to_update'] = (Prjts['updated_at']-Prjts['created_at']).dt.days
Prjts = Prjts.drop(labels=['needs_accomplish','needs_support','needs_value', 'partner_organization_id', 'match_job_id', 'agreed_at_community',
                          'publish_externally', 'enable_success_story', 'is_archived', 'share_metadata', 'campaign_id',
                          'image_id', 'admin_id', 'local_only', 'success_story', 'satisfaction_rating'],axis=1)
Prjts.head(10)

,id,organization_id,description,created_at,updated_at,state,user_id,project_inquiries_count,project_group,project_category_id,timeline,days_to_update
0,5948,4809,Website design optimization for new brand,2017-07-06 17:27:44.905846,2017-08-30 01:39:47.519662,closed,148960,3,NaN,19,2,54
1,5908,2051,New logo design,2017-06-28 16:55:47.852560,2017-11-11 18:17:38.379281,completed,122660,3,NaN,14,2,136
2,9339,1401,NaN,2019-01-15 00:00:34.636008,2019-01-15 00:00:34.636008,draft,169620,0,NaN,14,2,0
3,5975,4831,LOI and grant application,2017-07-10 15:36:19.184276,2017-09-27 13:39:17.766539,closed,149358,0,NaN,2,2,78
4,9981,7860,NaN,2019-04-03 03:19:57.848510,2019-04-03 03:19:57.848510,draft,172963,0,NaN,1,0,0
5,2646,515,Innovative marketing strategies for focused ca...,2016-04-12 17:27:07.249413,2016-10-31 17:44:05.193653,completed,3089,1,a,10,2,202
6,5803,4712,Design for outreach package,2017-06-14 22:13:48.016384,2017-08-04 19:43:00.130773,closed,148330,1,NaN,14,2,50
7,5367,4343,Public relations for special event,2017-04-12 16:24:51.181924,2017-09-22 11:43:53.799919,closed,145375,3,NaN,16,2,162
8,5905,4305,Logo and brand design for special event,2017-06-28 14:18:17.774979,2017-07-27 20:47:37.285743,closed,144980,3,NaN,14,2,29
9,2556,1921,NaN,2016-04-06 18:20:37.719807,2016-04-10 23:25:34.242759,rejected,109940,0,b,2,2,4


Now with our dataset reduced let's see just how many nulls there are in each column before we decide what to do with them.

In [10]:
Prjts.isnull().sum()

id                            0
organization_id               0
description                1675
created_at                    0
updated_at                    0
state                         0
user_id                       0
project_inquiries_count       0
project_group              9893
project_category_id           0
timeline                      0
days_to_update                0
dtype: int64

So it looks like most of our nulls are in project group, so let's go ahead and drop that as well.

In [11]:
Prjts = Prjts.drop(labels=['project_group'], axis=1)

In [12]:
Prjts['state'].unique()

array(['closed', 'completed', 'draft', 'rejected', 'reviewable',
       'deactivated', 'admin_close', 'archived', 'matched', 'applied',
       'published'], dtype=object)

In [13]:
mapping={
    'completed': 1,
    'matched': 1,
    'applied': 1,
    'published': 1
}
Prjts['succeeded'] = Prjts['state'].map(mapping) == 1
Prjts["succeeded"] = Prjts["succeeded"].astype(int)

In [14]:
Prjts.sort_values(by='id',axis=0).head()

,id,organization_id,description,created_at,updated_at,state,user_id,project_inquiries_count,project_category_id,timeline,days_to_update,succeeded
143,31,35,multipage brochure design,2014-08-19 14:53:23.622273,2015-02-20 19:59:24.590494,completed,1301,1,14,2,185,1
624,32,36,print design,2014-08-19 14:53:23.661564,2015-02-20 19:59:24.614798,completed,1302,1,14,2,185,1
837,33,37,NaN,2014-08-19 14:53:23.668844,2015-06-28 02:48:38.165214,completed,1303,1,20,2,312,1
859,34,477,Marketing consultation for campaign design,2014-08-19 14:53:23.683657,2015-02-20 19:59:24.628209,completed,1304,1,10,2,185,1
874,35,39,photography,2014-08-19 14:53:23.697066,2015-02-20 19:59:24.639884,closed,1305,0,13,2,185,0


In [15]:
answer_one = Prjts.groupby('project_category_id')['succeeded'].mean().sort_values(ascending=False)
answer_one = answer_one.to_frame()

In [16]:
answer_one

,succeeded
project_category_id,
15,0.591304
18,0.573614
6,0.566265
7,0.563694
12,0.524845
3,0.521739
14,0.508139
11,0.481153
10,0.460456


In [17]:
answer_three = Prjts.groupby('project_category_id')['days_to_update'].mean().sort_values(ascending=False)
answer_three = answer_three.to_frame()
answer_three

,days_to_update
project_category_id,
16,177.938124
10,161.095845
12,157.251553
2,153.723404
11,150.920177
18,147.845124
13,147.438003
6,146.321285
20,145.437374


# User_Inqry

In [18]:
#exploring User_Inqry dataset
User_Inqry.head()

,id,user_id,project_id,qualifications,created_at,updated_at,state,time_slots,scheduled_for,decision_deadline,conference_line_id,hours,pbc_rating,npo_rating,satisfaction_rating,pbc_review,archived
0,47,62,43,"I don't know that I am ""qualified"" since I exp...",2014-09-30 18:49:54.871459,2014-10-11 15:00:26.825409,rejected,"{""2014-10-06 15:00:00"",""2014-10-08 15:00:00""}",2014-10-08 15:00:00,2014-10-11 15:00:00,1.0,NaN,NaN,NaN,NaN,NaN,f
1,1170,2288,488,I would like to express my strong interest in ...,2015-03-25 23:40:58.903803,2015-03-31 20:38:40.241074,rejected,"{""2015-03-31 18:00:00"",""2015-04-01 18:00:00""}",2015-03-31 18:00:00,2015-04-07 18:00:00,3.0,NaN,NaN,NaN,NaN,NaN,f
2,2921,6404,1164,I have over 25 years of experience in market r...,2015-09-12 20:07:13.798524,2016-02-15 23:15:45.417164,rejected,"{""2015-09-15 14:00:00"",""2015-09-16 13:00:00"",""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
3,2414,5086,799,"Hello, Ms. Williams\r\n\r\nMy name is Michelle...",2015-07-22 19:27:43.237188,2015-09-22 13:54:33.762885,pbc_expired,"{""2015-08-25 14:00:00"",""2015-08-25 15:00:00""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
4,4037,9267,1666,"Hello,\n\nAm leonard and have over twenty year...",2016-01-14 15:50:29.750774,2016-01-20 20:30:08.534114,rejected,"{""2016-01-19 20:00:00"",""2016-01-20 20:00:00"",""...",2016-01-20 20:00:00,2016-01-27 20:00:00,3.0,NaN,NaN,NaN,NaN,NaN,f


In [19]:
User_Inqry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25084 entries, 0 to 25083
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   25084 non-null  int64  
 1   user_id              25084 non-null  int64  
 2   project_id           25084 non-null  int64  
 3   qualifications       25082 non-null  object 
 4   created_at           25084 non-null  object 
 5   updated_at           25084 non-null  object 
 6   state                25084 non-null  object 
 7   time_slots           25084 non-null  object 
 8   scheduled_for        15500 non-null  object 
 9   decision_deadline    12526 non-null  object 
 10  conference_line_id   13390 non-null  float64
 11  hours                1303 non-null   float64
 12  pbc_rating           924 non-null    float64
 13  npo_rating           702 non-null    float64
 14  satisfaction_rating  1061 non-null   float64
 15  pbc_review           375 non-null   

In [20]:
User_Inqry.isnull().sum()

id                         0
user_id                    0
project_id                 0
qualifications             2
created_at                 0
updated_at                 0
state                      0
time_slots                 0
scheduled_for           9584
decision_deadline      12558
conference_line_id     11694
hours                  23781
pbc_rating             24160
npo_rating             24382
satisfaction_rating    24023
pbc_review             24709
archived                   0
dtype: int64

### Thoughts on this dataset:
1) Again, we can reduce the size here.
2) Also a lot of nulls but some of them are useful to help answer the questions we have such as hours or satisfaction rating to give some insights on volunteers on projects.
3) Not a question we are answering but I think it would be interesting to see what the rejection rate is for volunteers. We could go a step further and see what it is by project category.

In [21]:
User_Inqry = User_Inqry.drop(labels=['scheduled_for', 'decision_deadline', 'conference_line_id', 'hours', 'pbc_rating',
                                    'npo_rating', 'satisfaction_rating', 'pbc_review', 'archived', 'time_slots'], axis=1)
User_Inqry['created_at'] = pd.to_datetime(User_Inqry['created_at'], format="%Y-%m-%d %H:%M:%S.%f")
User_Inqry['updated_at'] = pd.to_datetime(User_Inqry['updated_at'], format="%Y-%m-%d %H:%M:%S.%f")
User_Inqry['days_to_update'] = (User_Inqry['updated_at']-User_Inqry['created_at']).dt.days

In [22]:
User_Inqry['state'].unique()

array(['rejected', 'pbc_expired', 'npo_expired', 'cancelled', 'completed',
       'failed', 'accepted', 'admin_close', 'npo_rescheduled',
       'confirmed', 'applied', 'pbc_rescheduled', 'missed'], dtype=object)

In [23]:
mapping={
    'rejected': 'inqry_failed',
    'pbc_expired': 'inqry_failed',
    'npo_expired': 'inqry_failed',
    'cancelled': 'inqry_failed',
    'failed': 'inqry_failed',
    'admin_close': 'inqry_failed',
    'missed': 'inqry_failed'
}
User_Inqry['inqry_failed'] = User_Inqry['state'].map(mapping) == 'inqry_failed'
User_Inqry["inqry_failed"] = User_Inqry["inqry_failed"].astype(int)

In [24]:
User_Inqry.tail()

,id,user_id,project_id,qualifications,created_at,updated_at,state,days_to_update,inqry_failed
25079,26952,178353,11395,"I have design, content generation, and proofre...",2019-11-20 22:00:51.252953,2019-12-11 19:59:09.406476,pbc_expired,20,1
25080,26920,4172,11396,I live in Silicon Valley and work in tech as w...,2019-11-19 12:00:24.619799,2019-11-28 00:35:09.857962,accepted,8,0
25081,27064,182933,11450,I began my post-graduate professional career w...,2019-11-28 04:14:16.024452,2019-11-28 22:11:00.901167,rejected,0,1
25082,27033,174579,4129,I am an integrated communications expert with ...,2019-11-25 23:09:52.848121,2019-11-28 23:42:01.560910,accepted,3,0
25083,27069,182947,11796,"Hello,\nMy background is in Business Analysis ...",2019-11-28 17:41:20.908411,2019-12-10 03:57:53.690976,accepted,11,0


In [25]:
User_Inqry = User_Inqry.sort_values(by='id',ascending=True)

In [26]:
User_Inqry = User_Inqry.drop(labels=['created_at', 'updated_at'], axis=1)

In [27]:
User_Inqry = User_Inqry.merge(Prjts[['id','project_category_id']],left_on = 'project_id',right_on='id')
User_Inqry.head()

,id_x,user_id,project_id,qualifications,state,days_to_update,inqry_failed,id_y,project_category_id
0,1,23,52,I might be able to help with some photos. Not ...,npo_expired,200,1,52,13
1,580,1,52,data backfill,completed,681,0,52,13
2,2,24,51,Currently I am working for a company that deal...,rejected,178,1,51,14
3,80,256,51,Here’s Why I Sincerely Believe I Might Be Just...,npo_expired,118,1,51,14
4,162,345,51,\r\nI am an experienced Graphic Artist and Cre...,npo_expired,108,1,51,14


In [28]:
Days_to_match = User_Inqry.groupby('project_category_id')['days_to_update'].mean().sort_values(ascending=False)
Days_to_match = Days_to_match.to_frame()

In [29]:
Days_to_match = Days_to_match.rename(columns={'days_to_update':'avg_days_to_match'})
Days_to_match

,avg_days_to_match
project_category_id,
11,89.426528
12,83.620513
20,75.350536
13,71.980315
8,70.901257
10,70.581710
5,70.121457
14,69.884009
6,69.854545


In [30]:
Inqry_Recieved = User_Inqry.groupby('project_category_id')['id_x'].count().sort_values(ascending=False)
Inqry_Recieved = Inqry_Recieved.to_frame()
Inqry_Recieved = Inqry_Recieved.rename(columns={'id_x':'Num_Inqry_Recieved'})
Inqry_Recieved

,Num_Inqry_Recieved
project_category_id,
10,3029
14,2664
2,2264
18,2166
20,2054
19,1885
15,1471
1,1432
7,1173


In [31]:
Match_Rate = User_Inqry.groupby('project_category_id')['inqry_failed'].mean().sort_values(ascending=False)
Match_Rate = Match_Rate.to_frame()
Match_Rate

,inqry_failed
project_category_id,
18,0.842567
4,0.817610
19,0.810080
7,0.804774
1,0.801676
17,0.799629
20,0.796981
13,0.793963
9,0.781250


In [32]:
#joining all the answer files
Match_Rate.to_csv(os.path.join(path, 'Match_Rate.csv'))
Inqry_Recieved.to_csv(os.path.join(path, 'Inqry_Recieved.csv'))
Days_to_match.to_csv(os.path.join(path, 'Days_to_match.csv'))
answer_three.to_csv(os.path.join(path, 'answer_three.csv'))
answer_one.to_csv(os.path.join(path, 'answer_one.csv'))

# Session

In [33]:
# Exploring Session dataset
Session.sort_values(by='id',axis=0).head()

,id,created_at,updated_at,state,description,consultant_id,nonprofit_id,scheduled_for,time_slots,organization_id,conference_line_id,project_category_id,partner_organization_id,archived
38,34,2015-11-09 23:18:42.717496,2015-11-16 16:00:01.887738,completed,test,6.0,5118,2015-11-16 14:00:00,"{""2015-11-16 14:00:00"",""2015-11-17 17:00:00"",""...",239,1.0,20.0,NaN,f
36,35,2015-12-17 21:33:33.627942,2016-03-28 19:34:38.257339,cancelled,Our communications team did not develop our cu...,NaN,5062,NaN,"{""2015-12-21 14:00:00"",""2015-12-22 17:00:00"",""...",577,NaN,20.0,NaN,f
37,36,2015-12-21 20:32:17.550691,2016-03-28 19:35:04.089662,cancelled,Our communications team of two people needs he...,NaN,5062,NaN,"{""2015-12-30 17:00:00"",""2015-12-30 18:00:00"",""...",577,NaN,20.0,NaN,f
28,37,2016-01-21 01:39:13.613789,2016-03-28 19:35:23.081839,cancelled,I have a new creative way to make it easy for ...,NaN,9497,NaN,"{""2016-01-29 16:00:00"",""2016-01-29 19:00:00"",""...",2455,NaN,11.0,NaN,f
44,38,2016-01-21 01:43:08.028323,2016-02-01 21:00:01.402058,completed,I want to ensure that my website is conveying ...,9321.0,9497,2016-02-01 19:00:00,"{""2016-02-01 15:00:00"",""2016-02-01 19:00:00"",""...",2455,1.0,15.0,NaN,f


In [34]:
Session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1665 entries, 0 to 1664
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1665 non-null   int64  
 1   created_at               1665 non-null   object 
 2   updated_at               1665 non-null   object 
 3   state                    1665 non-null   object 
 4   description              1664 non-null   object 
 5   consultant_id            623 non-null    float64
 6   nonprofit_id             1665 non-null   int64  
 7   scheduled_for            576 non-null    object 
 8   time_slots               1665 non-null   object 
 9   organization_id          1665 non-null   int64  
 10  conference_line_id       625 non-null    float64
 11  project_category_id      1660 non-null   float64
 12  partner_organization_id  0 non-null      float64
 13  archived                 1665 non-null   object 
dtypes: float64(4), int64(3),

In [35]:
Session.isnull().sum()

id                            0
created_at                    0
updated_at                    0
state                         0
description                   1
consultant_id              1042
nonprofit_id                  0
scheduled_for              1089
time_slots                    0
organization_id               0
conference_line_id         1040
project_category_id           5
partner_organization_id    1665
archived                      0
dtype: int64

### Thoughts on this dataset:
1) Reduce the size of the dataset
2) Convert data type to datetime
3) Drop columns with null values

In [36]:
Session = Session.drop(labels=['time_slots','scheduled_for','consultant_id', 'conference_line_id', 
                               'nonprofit_id', 'partner_organization_id', 'archived'],axis=1)
Session['created_at'] = pd.to_datetime(Session['created_at'], format="%Y-%m-%d %H:%M:%S.%f")
Session['updated_at'] = pd.to_datetime(Session['updated_at'], format="%Y-%m-%d %H:%M:%S.%f")
Session['days_to_update'] = Session['updated_at']-Session['created_at']
Session['inqry_failed'] = Session['state'].map(mapping) == 'inqry_failed'

In [37]:
Session.head()

,id,created_at,updated_at,state,description,organization_id,project_category_id,days_to_update,inqry_failed
0,102,2017-01-10 19:04:55.570931,2017-01-10 19:05:13.719726,cancelled,test.,239,20.0,0 days 00:00:18.148795,True
1,110,2017-03-29 14:36:50.900996,2017-03-31 18:00:00.402136,completed,We're creating new websites for a couple of ou...,4218,20.0,2 days 03:23:09.501140,False
2,60,2016-03-17 16:08:47.426778,2016-03-30 14:52:12.539138,cancelled,The TASH website is a Wordpress website that w...,2153,20.0,12 days 22:43:25.112360,True
3,95,2016-08-24 19:37:31.043736,2016-09-06 15:00:01.828814,completed,We want to increase our online sales of tea an...,2933,10.0,12 days 19:22:30.785078,False
4,53,2016-03-17 00:05:42.531388,2016-03-24 16:00:01.854188,completed,We are currently in the process of redesigning...,1370,20.0,7 days 15:54:19.322800,False


## Pulling it all together
Now we have clean data sets that we can work with. Next, we want to pull some summary statistics and join some of the data sets to see if we can answer our business questions and develop some visualizations. 
Our business questions are: 
1) Which project types produce the best outcomes in survey data? 
2) Which project types perform better on the platform? Match rate, days to match, # of inquiries received. 
3) Which project types take the longest to complete? Shortest?

In [38]:
Session.to_csv(os.path.join(path, 'Session.csv'))
User_Inqry.to_csv(os.path.join(path, 'User_Inqry.csv'))
Prjts.to_csv(os.path.join(path, 'Prjts.csv'))
Prjt_Cat.to_csv(os.path.join(path, 'Prjt_Cat.csv'))

',id,group_slug,name\r\n0,1,business,Accounting & Finance\r\n1,3,business,Evaluation\r\n2,5,business,Program Design\r\n3,6,business,Research\r\n4,7,hr,HR Management\r\n5,8,hr,Board Development\r\n6,9,hr,Staff Development\r\n7,11,marketing,Brand Development\r\n8,12,marketing,Messaging\r\n9,13,marketing,Multimedia\r\n10,14,marketing,Design\r\n11,15,marketing,Copy writing/editing\r\n12,16,marketing,Public Relations\r\n13,18,it,CRM\r\n14,19,it,Website design\r\n15,21,it,Mobile Development\r\n16,22,business,Grant writing/development\r\n17,10,marketing,Marketing Strategy\r\n18,4,business,Project Management\r\n19,20,it,Website development\r\n20,17,it,IT Infrastructure\r\n21,2,business,Business Planning\r\n'

# Step 3
Now, we will switching to tableau to develop visualizations.